In [2]:
# Imports 
import math
import matplotlib.pyplot as plt
import numpy as np

# DUDC : discrete unit disc cover
## I) - DUDC dans $\mathbb{R} (d=1) $ 
### Question 01 : Algorithme glouton qui résout le DUDC:
#### Classes utilisées 

In [4]:
# Discrete Unit Disk Problem (DUDC)
# Used functions
def norm2(a,b):
    return (a.x-b.x)**2 + (a.y-b.y)**2

# Used Classes !
class Point:
    def __init__(self,x=0,y=0):
        self.x = x
        self.y=y
    def __repr__(self):
        return f'({self.x},{self.y})'
    def __lt__(self, other):
        return (self.x < other.x) or (self.x == other.x and self.y < other.y)

class Circle:
    def __init__(self,x,y,r=1):
        self.x=x
        self.y=y
        self.r=r
    def __repr__(self):
        return f'({self.x},{self.y})'
    def __lt__(self, other):
        return (self.x < other.x) or (self.x == other.x and self.y < other.y)
    def covers(self,point):
        return norm2(point,self) <= 1



## Classe DUDC qu'on va s'en servir pour la représentation graphique :

In [9]:
class DUDC:
    def __init__(self):
        self.P = set() # set of points
        self.Q = set() # set of unit circles
        self.num_P =self.num_Q= 0
        self.d = 1  # We are in |R
    def __repr__(self):
        return f'({self.P},{self.Q})'
    def figure(self,solution=[]):
        plt.clf()
        plt.axis("equal")
        for p in self.P:
            plt.scatter(p.x,p.y,marker='+',c='k',zorder=2)
        for q in self.Q:
           color = 'green' if q in solution else 'red'
           plt.scatter(q.x, q.y,marker = 'o',c=color, zorder = 1)
           circle = plt.Circle((q.x, q.y), 1, color = 'g', alpha = 0.1, zorder = 0)
           plt.gcf().gca().add_artist(circle)
        plt.xlabel('Abscisses X')
        plt.ylabel('Ordonnées Y')
        plt.title('DUDC Problem graph')
        return plt

True

# Algorithme Glouton (Greedy aproach) DUDC 1D

In [22]:
def dudc_1d(P,Q):
    points=sorted(list(P))
    circles=sorted(list(Q))
    Q_star=set()
    cpt_circle=0
    while len(points):
        p_left=points[0]
        # Le premier cercle qui couvre ce point
        for circle in circles:
            if circle.covers(p_left):
                break
            cpt_circle+=1

        if cpt_circle >= len(circles):
            print(' This DUDC problem hasn t any solution ')
            Q_star.clear()
            break

        # Le plus a droite cercle couvrant ce point
        while cpt_circle < len(circles) - 1 and circles[cpt_circle + 1].covers(p_left):
             cpt_circle += 1
             circle = circles[cpt_circle]
        Q_star.add(circle)
        for p in points:
            if circle.covers(p):
                points.remove(p)
    return Q_star




array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])